# ======================================
# Kaggle - Classificação Coluna Vertebral
# Modelos: KNN, Naïve Bayes, Decision Tree
# ======================================

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone  # Adicione esta importação


# ======================================
# 1. Carregamento dos dados
# ======================================

In [38]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Remover coluna de índice, se existir
if "Unnamed: 0" in train_df.columns:
    train_df = train_df.drop(columns=["Unnamed: 0"])
if "Unnamed: 0" in test_df.columns:
    test_df = test_df.drop(columns=["Unnamed: 0"])

# ======================================
# 2. Separar features e target
# ======================================

In [39]:
X = train_df.drop(columns=["class"])
y = train_df["class"]

# Codificar o alvo
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [40]:
# ======================================
# 2.5 ANALISAR DISTRIBUIÇÃO DAS CLASSES
# ======================================
print("Distribuição das classes no treino:")
print(pd.Series(y_encoded).value_counts())
print(f"Proporções: {pd.Series(y_encoded).value_counts(normalize=True).values}")

# Verificar se há desbalanceamento severo

Distribuição das classes no treino:
2    105
1     70
0     42
Name: count, dtype: int64
Proporções: [0.48387097 0.32258065 0.19354839]


In [41]:
# ======================================
# 2.6 ANÁLISE DETALHADA POR CLASSE
# ======================================
print("=" * 50)
print("ANÁLISE DETALHADA POR CLASSE:")

for class_idx, class_name in enumerate(label_encoder.classes_):
    class_mask = (y_encoded == class_idx)
    class_data = X[class_mask]
    
    print(f"\n--- {class_name} ---")
    print(f"Quantidade: {class_mask.sum()}")
    print("Estatísticas de degree_spondylolisthesis:")
    print(f"  Mínimo: {class_data['degree_spondylolisthesis'].min():.2f}")
    print(f"  Máximo: {class_data['degree_spondylolisthesis'].max():.2f}")
    print(f"  Média: {class_data['degree_spondylolisthesis'].mean():.2f}")
    print(f"  Mediana: {class_data['degree_spondylolisthesis'].median():.2f}")

ANÁLISE DETALHADA POR CLASSE:

--- Hernia ---
Quantidade: 42
Estatísticas de degree_spondylolisthesis:
  Mínimo: -10.68
  Máximo: 15.78
  Média: 2.27
  Mediana: 2.68

--- Normal ---
Quantidade: 70
Estatísticas de degree_spondylolisthesis:
  Mínimo: -11.06
  Máximo: 31.17
  Média: 2.34
  Mediana: 1.15

--- Spondylolisthesis ---
Quantidade: 105
Estatísticas de degree_spondylolisthesis:
  Mínimo: 1.01
  Máximo: 418.54
  Média: 54.81
  Mediana: 49.70


In [42]:
# ======================================
# 2.7 ANÁLISE DE DISTRIBUIÇÃO ENTRE TREINO E TESTE
# ======================================
print("=" * 50)
print("ANÁLISE DE DISTRIBUIÇÃO TREINO vs TESTE")

# Verificar se há diferenças significativas nas distribuições
print("\nEstatísticas das features no TREINO:")
print(X[['degree_spondylolisthesis', 'sacral_slope']].describe())

print("\nEstatísticas das features no TESTE:")
print(test_df[['degree_spondylolisthesis', 'sacral_slope']].describe())

# Verificar outliers extremos no teste
test_outliers = test_df[test_df['degree_spondylolisthesis'] > 100]  # Valores muito altos
if len(test_outliers) > 0:
    print(f"\n⚠️  ENCONTRADOS {len(test_outliers)} OUTLIERS NO TESTE:")
    print(test_outliers[['degree_spondylolisthesis', 'sacral_slope']])

ANÁLISE DE DISTRIBUIÇÃO TREINO vs TESTE

Estatísticas das features no TREINO:
       degree_spondylolisthesis  sacral_slope
count                217.000000    217.000000
mean                  27.715448     42.916988
std                   40.965137     13.631236
min                  -11.058179     13.516568
25%                    1.517203     33.366366
50%                   13.683047     42.447102
75%                   49.159426     51.766175
max                  418.543082    121.429566

Estatísticas das features no TESTE:
       degree_spondylolisthesis  sacral_slope
count                 93.000000     93.000000
mean                  22.986269     43.039797
std                   27.968445     12.996425
min                   -7.825986     13.366931
25%                    1.913307     33.091700
50%                   10.832011     42.137595
75%                   38.538741     53.530766
max                  148.753711     78.794052

⚠️  ENCONTRADOS 2 OUTLIERS NO TESTE:
    degree_spondylo

In [43]:
# ======================================
# 2.8 ANÁLISE DETALHADA DOS OUTLIERS E MUDANÇA DE DISTRIBUIÇÃO
# ======================================
print("=" * 50)
print("ANÁLISE DETALHADA DA MUDANÇA TEMPORAL")

# Analisar a distribuição por percentis
print("\nPercentis do degree_spondylolisthesis (Treino vs Teste):")
percentis = [0, 10, 25, 50, 75, 90, 95, 99, 100]
for p in percentis:
    treino_p = np.percentile(X['degree_spondylolisthesis'], p)
    teste_p = np.percentile(test_df['degree_spondylolisthesis'], p)
    print(f"P{p}: Treino={treino_p:.1f} vs Teste={teste_p:.1f}")

# O teste tem valores significativamente diferentes nos percentis altos

ANÁLISE DETALHADA DA MUDANÇA TEMPORAL

Percentis do degree_spondylolisthesis (Treino vs Teste):
P0: Treino=-11.1 vs Teste=-7.8
P10: Treino=-2.2 vs Teste=-1.7
P25: Treino=1.5 vs Teste=1.9
P50: Treino=13.7 vs Teste=10.8
P75: Treino=49.2 vs Teste=38.5
P90: Treino=69.4 vs Teste=56.7
P95: Treino=89.0 vs Teste=73.8
P99: Treino=123.9 vs Teste=105.5
P100: Treino=418.5 vs Teste=148.8


# ======================================
# 3. Padronização dos dados
# ======================================

In [44]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test_df)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [45]:
# ======================================
# 3.1 PRÉ-PROCESSAMENTO ROBUSTO
# ======================================
from sklearn.preprocessing import RobustScaler

print("=" * 50)
print("APLICANDO PRÉ-PROCESSAMENTO ROBUSTO")

# Usar RobustScaler que é menos sensível a outliers
robust_scaler = RobustScaler()
X_robust = robust_scaler.fit_transform(X)
test_robust = robust_scaler.transform(test_df)

# Aplicar winsorization para lidar com outliers extremos
def winsorize_features(X_array, limits=(0.01, 0.01)):
    """Aplica winsorization para limitar outliers"""
    from scipy.stats.mstats import winsorize
    X_winsorized = X_array.copy()
    for i in range(X_array.shape[1]):
        X_winsorized[:, i] = winsorize(X_array[:, i], limits=limits)
    return X_winsorized

# Aplicar apenas no degree_spondylolisthesis (que tem outliers extremos)
degree_idx = X.columns.get_loc('degree_spondylolisthesis')
X_robust[:, degree_idx] = winsorize_features(X_robust[:, degree_idx:degree_idx+1], limits=(0.05, 0.05)).flatten()
test_robust[:, degree_idx] = winsorize_features(test_robust[:, degree_idx:degree_idx+1], limits=(0.05, 0.05)).flatten()

print("Pré-processamento robusto aplicado com winsorization")

APLICANDO PRÉ-PROCESSAMENTO ROBUSTO


Pré-processamento robusto aplicado com winsorization


In [46]:
# ======================================
# 3.2 PRÉ-PROCESSAMENTO MAIS AGRESSIVO
# ======================================
print("=" * 50)
print("PRÉ-PROCESSAMENTO SUPER CONSERVADOR")

# Estratégia: Discretizar a feature problemática em bins
def discretize_degree_feature(X_array, test_array, n_bins=5):
    """Discretiza degree_spondylolisthesis para reduzir sensibilidade a outliers"""
    degree_values = np.concatenate([X_array, test_array])
    bins = np.percentile(degree_values, [0, 20, 40, 60, 80, 100])
    
    X_discrete = X_array.copy()
    test_discrete = test_array.copy()
    
    X_discrete = np.digitize(X_array, bins) - 1  # 0-indexed
    test_discrete = np.digitize(test_array, bins) - 1
    
    return X_discrete, test_discrete, bins

# Aplicar apenas na feature problemática
degree_idx = X.columns.get_loc('degree_spondylolisthesis')
X_degree = X_robust[:, degree_idx]
test_degree = test_robust[:, degree_idx]

X_degree_disc, test_degree_disc, bins = discretize_degree_feature(X_degree, test_degree, n_bins=4)

print(f"Bins criados: {bins}")
print(f"Distribuição treino discretizado: {np.bincount(X_degree_disc)}")
print(f"Distribuição teste discretizado: {np.bincount(test_degree_disc)}")

# Substituir nos arrays
X_conservative = X_robust.copy()
test_conservative = test_robust.copy()
X_conservative[:, degree_idx] = X_degree_disc
test_conservative[:, degree_idx] = test_degree_disc

PRÉ-PROCESSAMENTO SUPER CONSERVADOR
Bins criados: [-0.38161537 -0.27688602 -0.16383357  0.2859399   0.80169857  1.64442853]
Distribuição treino discretizado: [43 43 44 40 36 11]
Distribuição teste discretizado: [19 19 18 22 15]


# ======================================
# 4. Treinamento e Validação
# ======================================

In [47]:
# ======================================
# 4. Treinamento e Validação - SEM RANDOM FOREST
# ======================================

# ----- Modelo 1: KNN -----
print("=" * 50)
print("Treinando KNN...")
param_grid_knn = {
    'n_neighbors': [25, 27, 29, 31, 33, 35],
    'weights': ['uniform'],
    'metric': ['manhattan']
}
knn = KNeighborsClassifier()
grid_knn = GridSearchCV(knn, param_grid_knn, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
grid_knn.fit(X_scaled, y_encoded)
print("Melhores parâmetros KNN:", grid_knn.best_params_)
print("Acurácia KNN:", grid_knn.best_score_)

# ----- Modelo 2: Naïve Bayes -----
print("=" * 50)
print("Treinando Naïve Bayes...")
param_grid_nb = {
    'var_smoothing': [1e-7, 1e-6, 1e-5, 1e-4]
}
nb = GaussianNB()
grid_nb = GridSearchCV(nb, param_grid_nb, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
grid_nb.fit(X_scaled, y_encoded)
print("Melhores parâmetros Naïve Bayes:", grid_nb.best_params_)
print("Acurácia Naïve Bayes:", grid_nb.best_score_)

# ----- Modelo 3: Decision Tree -----
print("=" * 50)
print("Treinando Decision Tree...")
param_grid_dt = {
    'criterion': ['gini'],
    'max_depth': [2, 3],
    'min_samples_split': [10, 15, 20],
    'min_samples_leaf': [5, 7, 10],
    'max_features': ['sqrt'],
    'ccp_alpha': [0.01, 0.1, 1.0]
}

dt = DecisionTreeClassifier(random_state=42)
grid_dt = GridSearchCV(dt, param_grid_dt, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
grid_dt.fit(X_scaled, y_encoded)
print("Melhores parâmetros Decision Tree:", grid_dt.best_params_)
print("Acurácia Decision Tree:", grid_dt.best_score_)

# Comparação dos modelos
modelos_comparacao = {
    'KNN': grid_knn.best_score_,
    'Naïve Bayes': grid_nb.best_score_,
    'Decision Tree': grid_dt.best_score_
}

melhor_modelo_nome = max(modelos_comparacao, key=modelos_comparacao.get)
print(f"\nMelhor modelo: {melhor_modelo_nome} com acurácia: {modelos_comparacao[melhor_modelo_nome]:.4f}")

Treinando KNN...


Melhores parâmetros KNN: {'metric': 'manhattan', 'n_neighbors': 25, 'weights': 'uniform'}
Acurácia KNN: 0.8252642706131079
Treinando Naïve Bayes...
Melhores parâmetros Naïve Bayes: {'var_smoothing': 1e-07}
Acurácia Naïve Bayes: 0.8433403805496829
Treinando Decision Tree...
Melhores parâmetros Decision Tree: {'ccp_alpha': 0.01, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 7, 'min_samples_split': 10}
Acurácia Decision Tree: 0.7697674418604652

Melhor modelo: Naïve Bayes com acurácia: 0.8433


In [48]:
# ======================================
# 4.1 MODELAGEM COM VALIDAÇÃO TEMPORAL
# ======================================
print("=" * 50)
print("TREINAMENTO COM VALIDAÇÃO TEMPORAL")

# Ordenar os dados por degree_spondylolisthesis (simula ordenação temporal)
sorted_indices = np.argsort(X['degree_spondylolisthesis'])
X_sorted = X_robust[sorted_indices]
y_sorted = y_encoded[sorted_indices]

# Usar TimeSeriesSplit para validação mais realista
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)
print("Usando TimeSeriesSplit para validação mais realista")

# ----- Naïve Bayes com validação temporal -----
nb_temporal = GaussianNB(var_smoothing=1e-4)  # Mais regularização
scores_temporal = cross_val_score(nb_temporal, X_sorted, y_sorted, cv=tscv, scoring='accuracy')

print(f"Acurácia Naïve Bayes (validação temporal): {scores_temporal.mean():.4f}")
print(f"Desvio padrão: {scores_temporal.std():.4f}")

# Testar também com as 2 features mais importantes
X_important_temporal = X_robust[:, [X.columns.get_loc('degree_spondylolisthesis'), 
                                   X.columns.get_loc('sacral_slope')]]
scores_temporal_important = cross_val_score(nb_temporal, X_important_temporal, y_sorted, 
                                          cv=tscv, scoring='accuracy')

print(f"Acurácia com 2 features (temporal): {scores_temporal_important.mean():.4f}")

TREINAMENTO COM VALIDAÇÃO TEMPORAL
Usando TimeSeriesSplit para validação mais realista
Acurácia Naïve Bayes (validação temporal): 0.7000
Desvio padrão: 0.3055
Acurácia com 2 features (temporal): 0.3778


In [49]:
# ======================================
# 4.2 MODELO FINAL MELHORADO
# ======================================
print("=" * 50)
print("TREINAMENTO DO MODELO FINAL OTIMIZADO")

# Usar Random Forest simples mas eficaz
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=4,
    min_samples_split=15,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

# Usar todas as features mas com pré-processamento robusto
X_rf = X_conservative
test_rf = test_conservative

# Validação cruzada
scores_rf = cross_val_score(rf_model, X_rf, y_encoded, 
                          cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                          scoring='accuracy')

print(f"Acurácia Random Forest (5-fold CV): {scores_rf.mean():.4f} ± {scores_rf.std():.4f}")

# Treinar o modelo final
rf_model.fit(X_rf, y_encoded)

# Verificar importância das features
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("\nImportância das features no Random Forest:")
print(feature_importance_df.head(10))

TREINAMENTO DO MODELO FINAL OTIMIZADO
Acurácia Random Forest (5-fold CV): 0.8342 ± 0.0440

Importância das features no Random Forest:
                    Feature  Importance
5  degree_spondylolisthesis    0.357051
3              sacral_slope    0.208142
2     lumbar_lordosis_angle    0.141892
0          pelvic_incidence    0.140195
4             pelvic_radius    0.113927
1               pelvic_tilt    0.038793


In [50]:
# ========== ANÁLISE DE OVERFITTING - SEM RANDOM FOREST ==========
print("\n" + "="*50)
print("ANÁLISE DE OVERFITTING DETALHADA")

models = {
    'KNN': grid_knn.best_estimator_,
    'Naïve Bayes': grid_nb.best_estimator_, 
    'Decision Tree': grid_dt.best_estimator_
}

diferencas = {}
for name, model in models.items():
    model.fit(X_scaled, y_encoded)
    train_score = model.score(X_scaled, y_encoded)
    val_score = modelos_comparacao[name]
    diferenca = train_score - val_score
    diferencas[name] = diferenca
    print(f"{name} - Treino: {train_score:.4f}, Validação: {val_score:.4f}, Diferença: {diferenca:.4f}")

# Selecionar modelo com melhor trade-off
scores_finais = {}
for modelo in modelos_comparacao:
    penalty = diferencas[modelo] * 0.8
    scores_finais[modelo] = modelos_comparacao[modelo] - penalty

melhor_modelo_final = max(scores_finais, key=scores_finais.get)
print(f"\n=== MODELO SELECIONADO ===")
print(f"Modelo: {melhor_modelo_final}")
print(f"Acurácia validação: {modelos_comparacao[melhor_modelo_final]:.4f}")
print(f"Overfitting: {diferencas[melhor_modelo_final]:.4f}")
print(f"Score final: {scores_finais[melhor_modelo_final]:.4f}")

# Garantir overfitting baixo
if diferencas[melhor_modelo_final] > 0.05:
    print(f"\n⚠️  ALERTA: Overfitting alto ({diferencas[melhor_modelo_final]:.4f}).")
    melhor_modelo_final = min(diferencas, key=diferencas.get)
    print(f"Usando instead: {melhor_modelo_final} (overfitting: {diferencas[melhor_modelo_final]:.4f})")


ANÁLISE DE OVERFITTING DETALHADA
KNN - Treino: 0.8433, Validação: 0.8253, Diferença: 0.0181
Naïve Bayes - Treino: 0.8710, Validação: 0.8433, Diferença: 0.0276
Decision Tree - Treino: 0.7788, Validação: 0.7698, Diferença: 0.0090

=== MODELO SELECIONADO ===
Modelo: Naïve Bayes
Acurácia validação: 0.8433
Overfitting: 0.0276
Score final: 0.8212


# ======================================
# 5. Escolher o melhor modelo e treinar
# ======================================

In [51]:
# ======================================
# 5. Escolher o melhor modelo e treinar
# ======================================

# Selecionar o melhor modelo com base na análise de overfitting
if melhor_modelo_final == 'KNN':
    melhor_modelo = grid_knn.best_estimator_
elif melhor_modelo_final == 'Naïve Bayes':
    melhor_modelo = grid_nb.best_estimator_
elif melhor_modelo_final == 'Decision Tree':
    melhor_modelo = grid_dt.best_estimator_

# Treinar o modelo selecionado
melhor_modelo.fit(X_scaled, y_encoded)

print(f"\nModelo selecionado para submissão: {type(melhor_modelo).__name__}")


Modelo selecionado para submissão: GaussianNB


In [52]:
# ======================================
# 5.1 Seleção de Features OTIMIZADA
# ======================================

print("\n" + "="*50)
print("SELEÇÃO OTIMIZADA DE FEATURES")

# Usar apenas as 2 features mais importantes
top_2_features = ['degree_spondylolisthesis', 'sacral_slope']
print(f"Usando apenas 2 features: {top_2_features}")

X_important = X[top_2_features]
X_important_scaled = scaler.fit_transform(X_important)
test_important_scaled = scaler.transform(test_df[top_2_features])

# Re-treinar o melhor modelo com 2 features
melhor_modelo_simplificado = clone(models[melhor_modelo_final])
scores_simplificado = cross_val_score(melhor_modelo_simplificado, X_important_scaled, y_encoded,
                                    cv=cv_strategy, scoring='accuracy')

print(f"Acurácia com 2 features: {scores_simplificado.mean():.4f}")

# Treinar final
melhor_modelo_simplificado.fit(X_important_scaled, y_encoded)

# Verificar overfitting
train_score_simple = melhor_modelo_simplificado.score(X_important_scaled, y_encoded)
val_score_simple = scores_simplificado.mean()

print(f"Overfitting com 2 features: {train_score_simple - val_score_simple:.4f}")

# Decidir qual modelo usar
if (train_score_simple - val_score_simple) < (diferencas[melhor_modelo_final] * 0.7):
    print("✅ Usando modelo simplificado com 2 features")
    modelo_final = melhor_modelo_simplificado
    features_finais = top_2_features
    X_final_scaled = X_important_scaled
    test_final_scaled = test_important_scaled
    acuracia_esperada = val_score_simple
else:
    print("✅ Usando modelo completo")
    modelo_final = models[melhor_modelo_final]
    features_finais = X.columns.tolist()
    X_final_scaled = X_scaled
    test_final_scaled = test_scaled
    acuracia_esperada = modelos_comparacao[melhor_modelo_final]


SELEÇÃO OTIMIZADA DE FEATURES
Usando apenas 2 features: ['degree_spondylolisthesis', 'sacral_slope']
Acurácia com 2 features: 0.8252
Overfitting com 2 features: 0.0043
✅ Usando modelo simplificado com 2 features


In [53]:
# ======================================
# 5.2 ENSEMBLE SIMPLES PARA MELHOR GENERALIZAÇÃO
# ======================================
print("=" * 50)
print("IMPLEMENTANDO ENSEMBLE CONSERVADOR")

# Criar ensemble de modelos conservadores
from sklearn.ensemble import VotingClassifier

# Modelos muito conservadores
model1 = GaussianNB(var_smoothing=1e-4)
model2 = DecisionTreeClassifier(max_depth=2, min_samples_leaf=10, random_state=42)
model3 = KNeighborsClassifier(n_neighbors=35, weights='uniform', metric='manhattan')

ensemble = VotingClassifier(
    estimators=[
        ('nb', model1),
        ('dt', model2),
        ('knn', model3)
    ],
    voting='soft'  # Voting suave para probabilidades
)

# Validação do ensemble
scores_ensemble = cross_val_score(ensemble, X_robust, y_encoded, 
                                cv=cv_strategy, scoring='accuracy')

print(f"Acurácia do Ensemble: {scores_ensemble.mean():.4f}")

# Treinar ensemble final
ensemble.fit(X_robust, y_encoded)

IMPLEMENTANDO ENSEMBLE CONSERVADOR


Acurácia do Ensemble: 0.8574


VotingClassifier(estimators=[('nb', GaussianNB(var_smoothing=0.0001)),
                             ('dt',
                              DecisionTreeClassifier(max_depth=2,
                                                     min_samples_leaf=10,
                                                     random_state=42)),
                             ('knn',
                              KNeighborsClassifier(metric='manhattan',
                                                   n_neighbors=35))],
                 voting='soft')

In [54]:
# ======================================
# 5.4 VALIDAÇÃO COM HOLDOUT CONSERVADOR
# ======================================
print("=" * 50)
print("VALIDAÇÃO FINAL COM HOLDOUT")

# Criar um holdout conservador (20% dos dados)
from sklearn.model_selection import train_test_split

X_holdout, X_val, y_holdout, y_val = train_test_split(
    X_rf, y_encoded, test_size=0.2, 
    stratify=y_encoded, random_state=42
)

# Treinar no holdout
rf_model.fit(X_holdout, y_holdout)

# Prever no validation set
val_preds = rf_model.predict(X_val)
val_accuracy = np.mean(val_preds == y_val)

print(f"Acurácia no holdout validation: {val_accuracy:.4f}")

# Se a acurácia for boa, usar este modelo
if val_accuracy >= 0.80:
    print("✅ Modelo com boa performance no holdout")
    final_model = rf_model
    test_data = test_rf
else:
    print("⚠️  Performance no holdout baixa. Usando modelo anterior...")
    final_model = rf_model  # Usar mesmo assim
    test_data = test_rf

VALIDAÇÃO FINAL COM HOLDOUT
Acurácia no holdout validation: 0.8864
✅ Modelo com boa performance no holdout


# ======================================
# 6. Gerar previsão no conjunto de teste
# ======================================

In [ ]:
# ======================================
# 6. Gerar previsão no conjunto de teste - VERSÃO CORRIGIDA
# ======================================

print(f"\n=== PERFORMANCE FINAL ===")
print(f"Acurácia no treino: {train_score_simple:.4f}")
print(f"Acurácia na validação: {val_score_simple:.4f}")
print(f"Diferença (overfitting): {train_score_simple - val_score_simple:.4f}")

# Fazer previsões com o modelo final
predictions = modelo_final.predict(test_final_scaled)
pred_labels = label_encoder.inverse_transform(predictions)

# Criar arquivo de submissão
submission = sample_submission.copy()
submission["class"] = pred_labels
submission.to_csv("submission.csv", index=False)

print(f"\nArquivo 'submission.csv' gerado com sucesso!")
print(f"Modelo utilizado: {type(modelo_final).__name__}")
print(f"Features utilizadas: {features_finais}")
print(f"Acurácia esperada: {acuracia_esperada:.4f}")

GERANDO SUBMISSÃO FINAL COM AJUSTE


NameError: name 'test_final' is not defined